# FEDDYN BENCHMARK

We try to reproduce the results of the paper [FEDERATED LEARNING BASED ON  DYNAMIC REGULARIZATION](https://arxiv.org/abs/2111.04263) on the MNIST dataset. The paper introduces a new algorithm called FedDyn, which is a Federated Averaging algorithm where a linear and quadratic penalty terms are added to the loss, whose minima is consistent with the global stationary point. The algorithm is tested on the MNIST dataset with [MNIST_2NN](../fluke/nets.py:549)(a 2-layer MLP first introduced with FedAvg, in our case it's hidden layers will respectively be 200 and 100). The paper shows that FedProx outperforms FedAvg in terms of convergence speed and accuracy.

## Setup of the experiment

### Loading and splitting the dataset

In [ ]:
from fluke.data.datasets import Datasets
dataset = Datasets.get("mnist", path="../data", channel_dim=1)  #by default we use the data folder that will be created upon the first run, 
                                                                #the get method will create another folder if the selected dataset is not present

In [ ]:
from fluke.data import DataSplitter
splitter = DataSplitter(dataset=dataset,
                        distribution="iid",
                        client_split=0.1,
                        sampling_perc=1)

### Setting up the evaulator

In [ ]:
from fluke.evaluation import ClassificationEval, Evaluator
from fluke import GlobalSettings

evaluator = ClassificationEval(1,n_classes=dataset.num_classes)
GlobalSettings().set_evaluator(evaluator)
GlobalSettings().set_device("cuda")
GlobalSettings().set_seed(87)

### Setting hyperparameters and model   

In [ ]:
from fluke import DDict
from fluke import nets
# We set up the hyperparameters according to the paper's description
client_hp = DDict(
    batch_size=50,
    local_epochs=50,
    loss="CrossEntropyLoss",
    alpha=0.01,#overall best obtained from grid search 
    optimizer=DDict(
      lr=0.1,
      weight_decay=0.0001),
    scheduler=DDict(
      gamma=1,
      step_size=1)
)

alg_hp = DDict(
    client = client_hp,
    server=DDict(weighted=False),
    model="MNIST_2NN")

In [ ]:
from fluke.algorithms.feddyn import FedDyn
algorithm = FedDyn(100, splitter, alg_hp)

### Setting up the logger

In [ ]:
from fluke.utils.log import Log
logger = Log()
algorithm.set_callbacks(logger)

## Running the experiment   

In [ ]:
import torch

torch.use_deterministic_algorithms(mode=True, warn_only=True)
algorithm.run(100, 0.1)

: 

target 98.25~ ottenuto 92~ 